# Информационный поиск

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://nlp.stanford.edu/~manning/xyzzy/Intro_Inform_Retrieval_Russian.pdf
* https://en.wikipedia.org/wiki/Jaccard_index
* https://en.wikipedia.org/wiki/TF-IDF
* https://en.wikipedia.org/wiki/Okapi_BM25

## Задачи для совместного разбора

1\. Дан корпус текстов. Построить прямой и обратный индексы для слов из текста

In [1]:
corpus = [
    "Первым специальным индексом для запросов с джокером общего вида является перестановочный индекс",
    "Методы усовершенствования индексов для расширения функциональных возможностей и повышения скорости поиска"
]

In [2]:
pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 41.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=49397c91d83a0809c927bd73ab790de21595ef517dcb2d4c0a41bf7d58161af4
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
from nltk import word_tokenize

In [4]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
index = {}
for idx, doc in enumerate(corpus):
  index[idx] = [
      morph.parse(w)[0].normal_form
      for w in word_tokenize(doc)
  ]

In [7]:
index

{0: ['первый',
  'специальный',
  'индекс',
  'для',
  'запрос',
  'с',
  'джокер',
  'общий',
  'вид',
  'являться',
  'перестановочный',
  'индекс'],
 1: ['метод',
  'усовершенствование',
  'индекс',
  'для',
  'расширение',
  'функциональный',
  'возможность',
  'и',
  'повышение',
  'скорость',
  'поиск']}

In [8]:
from collections import defaultdict
# inv_index = defaultdict(list)
inv_index = defaultdict(set)

for idx, tokens in index.items():
  for tok in tokens:
    inv_index[tok].add(idx)


In [9]:
inv_index

defaultdict(set,
            {'первый': {0},
             'специальный': {0},
             'индекс': {0, 1},
             'для': {0, 1},
             'запрос': {0},
             'с': {0},
             'джокер': {0},
             'общий': {0},
             'вид': {0},
             'являться': {0},
             'перестановочный': {0},
             'метод': {1},
             'усовершенствование': {1},
             'расширение': {1},
             'функциональный': {1},
             'возможность': {1},
             'и': {1},
             'повышение': {1},
             'скорость': {1},
             'поиск': {1}})

2\. Посчитать индекс Жаккара для предложений из заданного корпуса.

In [ ]:
(len(set(index[0]).intersection(index[1])), len(set(index[0]).union(index[1])))

(2, 20)

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Считайте тексты новостей из файла `news.json`. Назовём прямым индексом словарь, где ключами являются номера новостей, а значениями - списки нормализованных слов, входящих в эту новость. Номер новости определяется ее положением в файле. Назовём обратным индексом словарь, где ключами являются нормальные формы слов, а значениями - множества номеров новостей, которые содержат данное слово (не обязательно в нормальной форме). Постройте прямой и обратный индекс. Выведите из длины на экран. Выведите значения обратного индекса по ключу "москва".
- [x] Проверено на семинаре

In [ ]:
from string import punctuation

In [ ]:
punc = punctuation + '«' + '»'
punc

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~«»'

In [ ]:
import json

with open('news.json') as f:
  data = json.load(f)

def remove_punc(sent):
  for p in punc:
    sent = sent.replace(p, '')
  return sent

data = list(
    map(
        remove_punc, data
    )
)

In [ ]:
index = {}
for idx, doc in enumerate(data):
  index[idx] = [
      morph.parse(w)[0].normal_form
      for w in word_tokenize(doc)
  ]

In [ ]:
from collections import defaultdict

inv_index_list = defaultdict(list)
inv_index_set = defaultdict(set)

for idx, tokens in index.items():
  for tok in tokens:
    inv_index_list[tok].append(idx)
    inv_index_set[tok].add(idx)

<p class="task" id="2"></p>

2\. Для каждого документа из текста определите, как часто слова этого документа встречаются в заданном тексте, воспользовавшись обратным индексом. Выведите на экран текст топ-3 новостей, наиболее похожих по набору токенов на заданный текст. Выведите на экран текст найденных новостей и количество совпадающих токенов.
- [x] Проверено на семинаре

In [ ]:
from nltk import word_tokenize
from collections import Counter

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
example = "Жириновский предложил перенести столицу из Москвы"

In [ ]:
res = {}
for word in word_tokenize(example.lower()):
  word = morph.parse(word)[0].normal_form
  res[word] = inv_index_list[word]
flat_list = Counter([item for sublist in res.values() for item in sublist])
counts = flat_list.most_common(3)

In [ ]:
for x in counts:
  print(data[x[0]], x[1])

трамп предложил перенести новые санкции против россии из за дня народного единства  3
31 декабря владимир жириновский предложит имя своего преемника  2
жириновский предложил дать ремня создателям гачимучи каверов на известные песни  2


<p class="task" id="3"></p>

3\. Для заданного текста новости найдите топ-3 наиболее похожих новости, воспользовавшись коэффициентом Жаккара. Для определения количества общих слов в тексте используйте обратный индекс.  Выведите на экран текст найденных новостей и значение коэффициента Жаккара для каждой новости.

In [ ]:
example = "Жириновский предложил перенести столицу из Москвы"

In [ ]:
res = {}
for word in word_tokenize(example.lower()):
  word = morph.parse(word)[0].normal_form
  res[word] = inv_index_list[word]
flat_list = Counter([item for sublist in res.values() for item in sublist])

In [ ]:
ind_j = {}
for k, v in flat_list.items():
  c = v
  a = len(index[k])
  b = len(example.split())
  ind_j[k] = c / (a + b - c)

In [ ]:
from collections import OrderedDict

d_descending = OrderedDict(sorted(ind_j.items(),
                                  key=lambda kv: kv[1], reverse=True))

In [ ]:
from itertools import islice
top3 = dict(islice(d_descending.items(), 3))
top3

{9: 0.2, 270: 0.18181818181818182, 36: 0.18181818181818182}

In [ ]:
for k, v in top3.items():
  print(data[k], v)

трамп предложил перенести новые санкции против россии из за дня народного единства  0.2
токаев предложил исключить россию из состава снг  0.18181818181818182
международную универсиаду 2023 года перенесут из екатеринбурга 0.18181818181818182


<p class="task" id="4"></p>

4\. Реализуйте функцию для расчета TF-IDF.

![image.png](attachment:image.png)
![image-2.png](attachment:image-2.png)
![image-3.png](attachment:image-3.png)

Для расчета обратной частоты документа воспользуйтесь обратным индексом из предыдущей задачи. При расчете IDF добавьте сглаживание, прибавив единицу к числителю и знаменателю дроби.

In [ ]:
import numpy as np

In [ ]:
def tfidf(sentence, word):
  TF = Counter(sentence.split())[word] / len(sentence.split())
  IDF = np.log((len(data) + 1)/ (len(inv_index[word])+1))
  return TF * IDF

<p class="task" id="5"></p>

5\. Используя собственную реализацию TD-IDF, закодируйте новости. Результатом должна являться матрица `MxN`, где `M` - количество новостей в корпусе, `N` - размер обратного индекса. Выведите форму полученной матрицы на экран.

In [ ]:
data_parsed = []
for doc in data:
  st = ''
  for w in word_tokenize(doc):
    st += morph.parse(w)[0].normal_form + ' '
  data_parsed.append(st)

In [ ]:
data_parsed[:3]

['российский бюджет в март недополучить более 300 миллиард рубль нефтегазовый доход ',
 'банк россия решить снизить ключевой ставка с 20 до 17 ',
 'мыс идокопас в нато назвать первый цель для начало полномасштабный вторжение в россия ']

In [ ]:
mat = []
for sent in data_parsed:
  word_vec = []
  for word in inv_index_set.keys():
    word_vec.append(tfidf(sent, word))
  mat.append(word_vec)

In [ ]:
len(mat), len(mat[0])

(500, 2441)

<p class="task" id="6"></p>

6\. Для заданного текста новости найдите топ-3 наиболее похожих новости, воспользовавшись матрицей TD-IDF из предыдущего задания. Для определения схожести используйте функцию косинусного расстояния. Выведите на экран текст найденных новостей и значение метрики близости для каждой новости.

In [ ]:
mat = np.array(mat)

In [ ]:
example = "Жириновский предложил перенести столицу из Москвы"

In [ ]:
example_vec = []
st = ''
for w in word_tokenize(example.lower()):
  word = morph.parse(w)[0].normal_form
  st += word + ' '

for word in inv_index_set.keys():
  example_vec.append(tfidf(st, word))

In [ ]:
example_vec = np.array(example_vec)

In [ ]:
len(example_vec[example_vec != 0])

6

In [ ]:
similarity = example_vec @ mat.T / ((example_vec**2).sum()**0.5 * (mat**2).sum(axis=1)**0.5)

In [ ]:
similarity.shape

(500,)

In [ ]:
ind = np.argsort(-similarity)[:3]
ind

array([  9, 270,  36])

In [ ]:
for x in ind:
  print(data[x], similarity[x])

трамп предложил перенести новые санкции против россии из за дня народного единства  0.3535533905932738
токаев предложил исключить россию из состава снг  0.3086066999241838
международную универсиаду 2023 года перенесут из екатеринбурга 0.3086066999241838


<p class="task" id="7"></p>

7\. Для заданного текста новости найдите топ-3 наиболее похожих новости, воспользовавшись собственной реализацией функции ранжирования [BM25](https://en.wikipedia.org/wiki/Okapi_BM25). Выведите на экран текст найденных новостей и значение метрики ранжирования для каждой новости.

In [ ]:
example = "Жириновский предложил перенести столицу из Москвы"

In [ ]:
import math
def bm25(query, doc, n_docs, avg, inv_index, k=1.5, b=0.75):
    res = 0
    words = word_tokenize(doc.lower())
    C = Counter([
        morph.parse(word)[0].normal_form for word in words
    ])
    for q in word_tokenize(query.lower()):
        w = morph.parse(q)[0].normal_form
        IDF = math.log((n_docs - len(inv_index[w]) + 0.5) / (len(inv_index[w]) + 0.5) + 1)
        res += IDF * C[q]*(k + 1) / (C[q] + k*(1 - b + b*len(words)/avg))
    return res

In [ ]:
avg = sum([len(sent.strip().split()) for sent in data]) / len(data)
rating = []
for i, D in enumerate(data):
    rating.append(
        (i, bm25(example, D, len(data), avg, inv_index_set))
    )

In [ ]:
ans = sorted(rating, key=lambda x: x[1], reverse=True)[:3]
for t in ans:
    print(data[t[0]], t[1])

международную универсиаду 2023 года перенесут из екатеринбурга 8.420041222128095
трамп предложил перенести новые санкции против россии из за дня народного единства  6.75263295335603
жириновский пожаловалась на недовольных туристами жителей аляски 5.905186518576618


## Обратная связь
- [x] Хочу получить обратную связь по решению